# OPEN WEATHER ONE CALL API FOR WEATHER VARIABLES
---
---
### *Building out functions to extract OpenWeather data per trackpoint. Startup subscription level used.*

# Table of Contents
---

### [Sources ](#Sources)
### [Imports ](#Imports)
### [Historical Weather API Testing](#Testing:-Weather-from-OneCall-API)
### [Hourly Forecast API Testing](#Testing:-Hourly-48-Hour-Forecast)
### [Daily Current/Forecast API Testing](#Testing:-Current/7-Day-Forecast-Daily-Weather-from-OneCall-API)

# Sources
---

In [1]:
# https://openweathermap.org/api/one-call-3
# Lesson 5.01 - APIs
# https://note.nkmk.me/en/python-unix-time-datetime/

# Imports
---

In [2]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import requests

import datetime

%matplotlib inline
pd.set_option('display.max_columns', None)

In [3]:
API_key = 'd43015e86fe9a04995afe46ef9ad42c4'

# Testing: Weather from OneCall API
---

### "timemachine" allows timestamps within 5-day forecast

In [4]:
url_weather = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=38.77346634864807&lon=-121.36368582956493&dt=1658362065&appid={API_key}"
weather_req = requests.get(url_weather)
print(f'Status: {weather_req.status_code}')

weather = weather_req.json()
print(weather.keys())

weather_df = json_normalize(weather['data'])
weather_df = weather_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
       'dew_point', 'clouds', 'wind_speed', 'wind_deg']]

# Convert dt UNIX to datetime UTC
weather_df['dt'] = weather_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

weather_df.head()

Status: 200
dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'data'])


,dt,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg
0,2022-07-20 17:07:45,312.89,311.34,1014,17,283.27,0,3.6,220


In [5]:
weather

{'lat': 38.7735,
 'lon': -121.3637,
 'timezone': 'America/Los_Angeles',
 'timezone_offset': -25200,
 'data': [{'dt': 1658362065,
   'sunrise': 1658321791,
   'sunset': 1658374005,
   'temp': 312.89,
   'feels_like': 311.34,
   'pressure': 1014,
   'humidity': 17,
   'dew_point': 283.27,
   'clouds': 0,
   'visibility': 10000,
   'wind_speed': 3.6,
   'wind_deg': 220,
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}]}]}

## Function get_weather(unix_datetime, lat, lon)

In [6]:
def get_weather(unix_datetime, lat, lon, API_key= API_key):
    '''
    Using the OpenWeather OneCall 3.0 API to pull weather data by Unix timestamp
    Requires API key for at minimum Startup OpenWeather subscription
    Input: UNIX datetime for time requesting, latitude and longitude of trackpoint
    Output: Pandas dataframe for that datetime and location's weather
    '''
    
    url_point = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={unix_datetime}&appid={API_key}"
    point_req = requests.get(url_point)
    print(f'API Status: {point_req.status_code}')

    weather = point_req.json()

    weather_df = json_normalize(weather['data'])
    weather_df = weather_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
           'dew_point', 'clouds', 'wind_speed', 'wind_deg']]
    
    return weather_df

# Testing: Hourly 48-Hour Forecast
---

In [7]:
url_fore_hourly = f"https://api.openweathermap.org/data/3.0/onecall?lat=38.752125&lon=-121.288010&exclude=current,daily,minutely,alerts&appid={API_key}"
fore_hourly_req = requests.get(url_fore_hourly)
print(f'Status: {fore_hourly_req.status_code}')

wh_fore = fore_hourly_req.json()
print(wh_fore.keys())

wh_fore_df = json_normalize(wh_fore['hourly'])
wh_fore_df = wh_fore_df[['dt', 'pressure', 'humidity', 'dew_point',
       'clouds', 'wind_speed', 'wind_deg', 'wind_gust']]
wh_fore_df.shape

Status: 200
dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'hourly'])


(48, 8)

In [8]:
wh_fore_df.head()

,dt,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,wind_gust
0,1660345200,1013,19,281.43,0,2.60,200,3.23
1,1660348800,1013,22,283.57,0,2.93,200,3.35
2,1660352400,1013,20,282.12,0,3.84,202,3.55
3,1660356000,1012,19,280.56,0,4.21,200,5.01
4,1660359600,1012,22,280.35,0,3.74,188,6.69


In [9]:
def forecast_hourly_48(lat, lon, API_key = API_key):
    '''
    Using OpenWeather OneCall 3.0 API, takes in the latitude and longitude of a location and returns a dataframe with 48-hour forecast by hour.
    Requires API key for at minimum Startup OpenWeather subscription
    Input: latitude and longitude of a location
    Output: dataframe
    '''
    # get 48-hour forecast for location
    url_fore_hourly = f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude=current,daily,minutely,alerts&appid={API_key}"
    fore_hourly_req = requests.get(url_fore_hourly)
    print(f'Status: {fore_hourly_req.status_code}')

    wh_fore = fore_hourly_req.json()
    print(wh_fore.keys())
    
    # turn to dataframe
    wh_fore_df = json_normalize(wh_fore['hourly'])
    wh_fore_df = wh_fore_df[['dt', 'pressure', 'humidity', 'dew_point',
           'clouds', 'wind_speed', 'wind_deg', 'wind_gust']]
    
    return wh_fore_df

# Testing: Current/7-Day Forecast Daily Weather from OneCall API
---

In [10]:
url_current_daily = f"https://api.openweathermap.org/data/3.0/onecall?lat=38.752125&lon=-121.288010&exclude=current,minutely,hourly,alerts&appid={API_key}"
current_daily_req = requests.get(url_current_daily)
print(f'Status: {current_daily_req.status_code}')

wd_current = current_daily_req.json()
print(wd_current.keys())

wd_current_df = json_normalize(wd_current['daily'])
wd_current_df = wd_current_df[['dt', 'pressure', 'humidity', 'dew_point',
       'clouds', 'wind_speed', 'wind_deg', 'wind_gust']]

# TEMPERATURE - KELVINS
temp_list = []
for i in range(len(wd_current['daily'])):
    avg_temp = round(sum([value for value in wd_current['daily'][i]['temp'].values()])/len(wd_current['daily'][i]['temp']), 2)
    temp_list.append(avg_temp)
wd_current_df['temp_avg'] = temp_list

# FEELS LIKE - KELVINS
fl_list = []
for i in range(len(wd_current['daily'])):
    avg_fl = round(sum([value for value in wd_current['daily'][i]['feels_like'].values()])/len(wd_current['daily'][i]['feels_like']), 2)
    fl_list.append(avg_fl)
wd_current_df['feels_like_avg'] = fl_list

# Convert dt UNIX to datetime UTC
wd_current_df['datetime'] = wd_current_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

wd_current_df.head(10)

Status: 200
dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'daily'])


,dt,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,wind_gust,temp_avg,feels_like_avg,datetime
0,1660334400,1014,19,279.20,0,4.21,200,6.69,299.74,298.45,2022-08-12 13:00:00
1,1660420800,1014,13,274.91,0,3.77,205,6.44,300.21,298.58,2022-08-13 13:00:00
2,1660507200,1012,11,273.15,0,3.27,201,6.09,301.37,299.79,2022-08-14 13:00:00
3,1660593600,1008,10,274.22,0,3.20,205,6.16,303.41,301.64,2022-08-15 13:00:00
4,1660680000,1006,8,271.63,24,3.13,216,5.75,305.31,303.33,2022-08-16 13:00:00
5,1660766400,1007,13,279.43,51,3.40,188,6.00,306.29,304.21,2022-08-17 13:00:00
6,1660852800,1006,11,279.25,0,3.21,240,6.73,307.24,305.28,2022-08-18 13:00:00
7,1660939200,1005,10,277.30,0,4.48,201,7.85,306.72,304.55,2022-08-19 13:00:00


## Function curr_fore_weather_daily(lat, lon):

In [11]:
def curr_fore_weather_daily(lat, lon, API_key = API_key):
    '''
    Using OpenWeather OneCall 3.0 API, takes in the latitude and longitude of a location and returns a dataframe with current + 7-day forecast.
    Input: latitude and longitude of a location
    Output: dataframe
    '''
    
    url_current_daily = f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude=current,minutely,hourly,alerts&appid={API_key}"
    current_daily_req = requests.get(url_current_daily)
    print(f'Status: {current_daily_req.status_code}')

    wd_current = current_daily_req.json()
    print(wd_current.keys())

    wd_current_df = json_normalize(wd_current['daily'])
    wd_current_df = wd_current_df[['dt', 'pressure', 'humidity', 'dew_point',
           'clouds', 'wind_speed', 'wind_deg', 'wind_gust']]

    # TEMPERATURE - KELVINS
    temp_list = []
    for i in range(len(wd_current['daily'])):
        avg_temp = round(sum([value for value in wd_current['daily'][i]['temp'].values()])/len(wd_current['daily'][i]['temp']), 2)
        temp_list.append(avg_temp)
    wd_current_df['temp_avg'] = temp_list

    # FEELS LIKE - KELVINS
    fl_list = []
    for i in range(len(wd_current['daily'])):
        avg_fl = round(sum([value for value in wd_current['daily'][i]['feels_like'].values()])/len(wd_current['daily'][i]['feels_like']), 2)
        fl_list.append(avg_fl)
    wd_current_df['feels_like_avg'] = fl_list

    # Convert dt UNIX to datetime UTC
    wd_current_df['datetime'] = wd_current_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    
    return wd_current_df